# 2-Survey Design 1: Data preparation

In this exercise we are going to cover the following topics:
1. Download OpenStreetMap street data.
2. Clip street data to area of interest 
3. Reproject street data to UTM coordinate reference system.
4. Clean street data with GRASS v.clean.
5. Create a pgRouting enabled database and load street data to it.

We will need the following software for this exercise:
1. QGIS >2.0 with GRASS extension
2. PostgreSQL >9.1 with PostGIS >2.0 and pgRouting >2.4 extensions

We will download all the relevant datasets during this exercise.

Author: Marc Wieland, M. Pittore
Last modified: 15.05.2017

## Download OpenStreetMap street data

First we need our street data. OpenStreetMap is a valuable resource to get data. You can **download preprocessed OpenStreetMap** data directly as Shapefiles (.shp) from Geofabrik under the following link: http://download.geofabrik.de

In this exercise we will use a dataset for Lima, Peru: http://download.geofabrik.de/south-america/chile-latest-free.shp.zip

Download the data, extract it and **load the file *gis.osm_roads_free_1.shp* into QGIS**. The Shapefile contains the streets for the whole country.

## Clip street data to area of interest

Next, we need to clip the street data to our **Region Of Interest (ROI)**. In the exercise data folder you find a Shapefile called *santiago_roi.shp*. Load it to QGIS.

To **clip the street data to the AOI** go to *Vector/Geoprocessing Tools/Clip* in the main menu of QGIS and fill the context menu as follows:
    
    Input layer: gis.osm_roads_free_1.shp
    Clip layer: santiago_roi.shp
    Clipped: santiago_roads_wgs.shp
    
Run the tool by clicking *OK*.

## Reproject street data to UTM coordinate reference system

The downloaded street data are in WGS84 **Coordinate Reference System (CRS)**, which measures in degrees. For the routing we will need to compute costs in meters (e.g. for the shortest path). Therefore, it is better to transform our street data into a reference system that measures in meters before using it for routing. Here we will reproject the data into the respective UTM CRS (UTM 19S for Santiago de Chile).

To **reproject a vector dataset**, simply right mouse click on the layer *osm_streets* in the layers panel in QGIS and click on *Save as..*. In the context menu that opens fill the following fields:
    
    File name: santiago_roads_utm.shp
    CRS: WGS 84 / UTM zone 19S
    
Run the tool by clicking *OK*. This writes the outputs into a new file called *osm_streets_utm.shp*.

---
## CAUTION: it may take a long time!  
### (in case load the corresponding data **osm_streets_utm_clean.shp** which has been already cleaned)
---

## Clean street data with GRASS v.clean

Next, we need to make sure that the polylines in the street data have a break at every crossing. This mean that every crossing should have a vertice. This is important for making the dataset routable later on and ensures that at every street crossing a turn is possible (Fig. 1).

To **clean street data** we use the GRASS tool *v.clean*. You can find the tool in the processing toolbar in QGIS under the GRASS provider's group. Simply type *v.clean* into the search field at the top of the processing toolbar and double click on it to open the tool. In the context menu that opens fill the following fields:

    Layer to clean: santiago_roads_wgs.shp
    Cleaning tool: break
    Threshold: 0.100000
    Cleaned: santiago_roads_wgs_clean.shp
    
The other fields can be left in their default state. Run the tool by clicking *OK*. This writes the outputs into a new file called *santiago_roads_wgs_clean.shp*.


---

## Reproject street data to UTM coordinate reference system

The downloaded street data are in WGS84 **Coordinate Reference System (CRS)**, which measures in degrees. For the routing we will need to compute costs in meters (e.g. for the shortest path). Therefore, it is better to transform our street data into a reference system that measures in meters before using it for routing. Here we will reproject the data into the respective UTM CRS (UTM 19S for Santiago de Chile).

To **reproject a vector dataset**, simply right mouse click on the layer *osm_streets* in the layers panel in QGIS and click on *Save as..*. In the context menu that opens fill the following fields:
    
    File name: santiago_roads_clean_utm.shp
    CRS: WGS 84 / UTM zone 19S
    
Run the tool by clicking *OK*. This writes the outputs into a new file called *santiago_roads_clean_utm.shp*.

## Create a pgRouting enabled database and load street data to it

Next, we create a PostgreSQL database with PostGIS and pgRouting extensions enabled. pgRouting extends a PostgreSQL / PostGIS spatial database with routing functionality. More information can be found on here: http://pgrouting.org.

First of all we need to **create a new database** on the PostgreSQL cluster. For this open up a terminal shell window and type the following command to create a new database called *osmrouting* as user *postgres* in the database cluster on *localhost*:

    $ createdb -h localhost -p 5432 -U postgres osmrouting

To **enable PostGIS and pgRouting extensions** for the newly created database we can simply execute the *pgr_customfunctions.sql* script in the exercise folder. This script also adds some additional functions to the database that we will need later on for routing via multiple stops. Run the script in the shell:
    
    $ psql -h localhost -p 5432 -U postgres -d osmrouting -f pgr_customfunctions.sql


## Load some data into the database

Once the database with routing extension is created, we can **load our street data** to it. To do this we simply run the following command in the shell:

    ogr2ogr -f "PostgreSQL" PG:"host=localhost user=postgres dbname=osmrouting password=postgres" santiago_roads_clean_utm.shp '-lco  FID=gid -lco GEOMETRY_NAME=geom -nln public.santiago_roads 

### Note: if 'gid' field already exists in the dataset, remove the following option:    '-lco  FID=gid'

This connects to the *osmrouting* database on *localhost* as user *postgres* and loads the *osm_streets_utm_clean.shp* Shapefile into the table *public.osm_streets* with geometry column name *geom* and primary key on column *gid*. It also creates a spatial index on the geometry column for us, which is makes spatial queries on this table a lot faster.



## Note: You can load directly the loaded layer from the database into the QGIS system

### Caution: if you modify the loaded features in QGIS, and save them, the corresponding layer in the database willl be changed

---

# Generate a proxy to the urbanization

## Since the buildings set is quite incomplete, we will extract the road nodes and use them to generate a 2-dimensional density distribution

Open PGAdmin3 from the command bar, access the database osmrouting and open a SQL script window. 

type in the following query: 

```sql
create table road_nodes as
select st_intersection(a.geom,b.geom), count(distinct a.geom) 
from santiago_roads as a, santiago_roads as b
where st_touches(a.geom,b.geom) and a.gid < b.gid
group by st_intersection(a.geom,b.geom);
```

and press F5 to send the query.

---

Once the query returned successfully, we load the resulting table ** road_nodes ** in QGIS to review it.

And we save it to a shapefile example  ** road_nodes.shp **

---

Now we will apply the ** raster->heat map ** function of QGIS to the layer saved as to generate a spatial density. Let´s use the following values: 

2'000 layer unit


# Load the "integrated Risk" layer from the GEM project 

load the peru_vir_gem.shp file,

load (lima_roi.shp) or define a clipping region and clip the integrated risk layer to the lima ROI (region of interest) using the function 'clip' of QGIS->geoprocessing tools

### Caution: make sure the clipping layer has the same projection as the layer to be clipped

save the clipped vector layer to the file vir_clipped.shp


Now open up the "Prioritization_Focus_Map" notebook to continue with the focus map.


---

# Prioritization: see exercise on Focus Maps 

---

# Routing

We now can **load a set of sample points** to the database that we will use later on as route stops for the routing.

    $ ogr2ogr -f "PostgreSQL" PG:"host=localhost user=postgres dbname=osmrouting password=postgres" samples_srs.shp -lco GEOMETRY_NAME=geom -lco FID=gid -nln public.samples_srs